In [8]:
import os
import pdftotext
# I saved all PDF files under the PDF folder in my env directory
pdf_docs_path = os.path.join("PDF")
one_pdf_path = os.path.join(pdf_docs_path, document)

In [9]:
import pandas as pd
df = pd.DataFrame(columns=['File', 'Page', 'Text'])

In [10]:
# Now let's supercharge the damn thing
with os.scandir(pdf_docs_path) as entries:
    for entry in entries:
        if entry.name != ".DS_Store":
            print(entry.name)

ANINTROENGLISH_EC.PDF
ccp_impactonpeople.pdf
Adapting to Climate Change in Coastal Communities In Canada White Paper.pdf
Guidebook-2016.pdf
81363.pdf
protect-your-home-from-severe-wind-pdf.pdf
protect-your-home-from-snow-ice-storms.pdf
A_Residential_Guide_to_Flood_Prevention_and_Recovery.pdf
WCEL_climate_change_FINAL.pdf
adapt_bulletin-adapt1-eng.pdf
Report on Effects of a Changing Climate to the US Department of Defense.pdf
Vulnerability Guidebook_June2_EN.pdf
protect-your-home-from-wildfire.pdf
env-yukon-state-play-analysis-climate-change-impacts-adaptation.pdf
Windsor Climate Change Adaptation Plan.pdf
Vancouver-Climate-Change-Adaptation-Strategy-2012-11-07.pdf
public_guideline__principles_of_climate_adaptation_and_mitigation_for_engineers.pdf
Floodproofing.pdf
VOLUME 4 Facing Rising Waters 2018.pdf
Spring_Flood_Fact_Sheet.pdf
climate_data_discussion_primer.pdf
FloodRecovery-e.pdf
slr-primer.pdf
ClimatRisk-E-ACCESSIBLE.pdf
Synthesis_Eng.pdf
landuse-e.pdf
municipal-climate-change-act

### All NLP and other dependencies for Lambda Functions

In [11]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from collections import Counter
import string
import json
nlp = spacy.load("en_core_web_sm")

### Action extraction function

In [12]:
# Defining the imperative sentence detector for action extraction as well as geopolitical entities (GPE)
def impSentenceExtractor(someText):
    
    doc = nlp(someText)
    impSentList=[]
    gpeList=[]
    specialWords = ("always","never","please","don't","not")
    # Just a list of words not to be confused with places
    gpeStopWords = ("Wildfire", "Flood", "Flooding", "Hurricane", "Store", "Driveway", "Building", "Uncut", "Rbon", "Complementary")
    # Extract sentences from block of text
    for sentence in doc.sents:
        
        # Check if it is not an interrogative sentence:
        if sentence[len(sentence)-1].shape_ != "?":
        
            # Rule 1
            if sentence[0].tag_=='VB' and len(sentence) > 2 and not (sentence.text in impSentList):
                impSentList.append(sentence.text.capitalize())
                
            else:
                if sentence[0].tag_=='VBG' and len(sentence) > 2 and not (sentence.text in impSentList):
                    # Caveat to rule 1: search for another verb
                    addSentence = False
                    for token in sentence:
                        if token.tag_=='VB':
                            addSentence = True
                    if addSentence:
                        impSentList.append(sentence.text.capitalize()) 

                
                # Rules 2 & 3         
                else:
                    addSentence = False
                    for token in sentence:
                        if token.lower_ in specialWords:
                            n = token.i - sentence.start #find the position of the special word in the sentence
                            if sentence[n+1].pos_=='VERB' and sentence[n+1].tag_=="VB" and len(sentence) > 2: 
                                addSentence = True
                    
                    if addSentence and not (sentence.text in impSentList):
                        impSentList.append(sentence.text.capitalize())

    # Geopolitical entity extraction
    for ent in doc.ents:
        if ent.label_ == "GPE":
            # Append it if not already there:
            if not(ent.text in gpeList) and not (ent.text in gpeStopWords):
                gpeList.append(ent.text)
    return impSentList, gpeList

### Frequent words extractor function

In [13]:
# Defining a function for frequent word extraction, returning a simple string:
def frequentClimateWordsExtractor(text):

    # Dictionary of relevant words
    dictionary = ["snow","change","climate","heatwave","adaptation","tornado","water","icestorm","risk","impact","level","community","land","management","planning","development","http","plan","infrastructure","sea","event","action","vulnerability","flood","assessment","storm","temperature", "low","rise","resource","weather","strategy","damage","effect","precipitation","hazard","ice","protection","home","flooding","erosion","environment","emission","al","winter","heat","forest","wind","mitigation","emergency","coast","shoreline","greenhouse","elevation","carbon","wave","dike","wetland","disaster","conservation","reduction","fire","rain","drainage","ground","power","stormwater","roof","rainfall","extreme","wildfire","reference","vegetation","threat","drought","disease","coastline","sewer","nature","neutral","neutrality"]

    # Loads text with linguistic annotations from Spacy
    my_doc = nlp(text)

    filteredDoc = []
    filteredList = []
    
    # Returns a list with relevant words filtered by the dictionary
    for sentence in my_doc.sents:
        for word in sentence:
            if not(word.is_stop) and (word.pos_=='NOUN' or word.pos_=='PROPN'):
                filteredDoc.append(word.text.lower())

    nounsFreqDistribution = Counter(filteredDoc)
    
    listOfWords=""
    for word in nounsFreqDistribution.most_common(300):
        if word[0] in dictionary:
            listOfWords = listOfWords + word[0] + ", "
            filteredList.append(word[0])
    return listOfWords, filteredList

### Disaster Classifier (rules-based)

In [14]:
# Trying a simple rule-based approach to disaster classification
def disasterType(key_arr):
    
    # Undefined to start with
    # disaster_class = "Undefined"
    numDetected = 0
    disaster_class = []
    
    if "carbon" in key_arr and ("neutral" in key_arr or "neutrality" in key_arr):
        disaster_class.append("Carbon Neutrality")
        numDetected+=1
        
    if "adaptation" in key_arr and ("change" in key_arr or "plan" in key_arr):
        disaster_class.append("Climate Change Adaptation")
        numDetected+=1
        
    if "drought" in key_arr:
        disaster_class.append("Drought")
        numDetected+=1
        
    if "flood" in key_arr or "flooding" in key_arr or "rainfall" in key_arr or "stormwater" in key_arr or ("sea" in key_arr and "level" in key_arr and "rise" in key_arr):
        disaster_class.append("Flooding")
        numDetected+=1
        
    if ("heat" in key_arr and "extreme" in key_arr) or "heatwave" in key_arr:
        disaster_class.append("Heatwave")
        numDetected+=1
        
    if "mitigation" in key_arr:
        disaster_class.append("Mitigation")
        numDetected+=1
        
    if "wind" in key_arr or "tornado" in key_arr:
        disaster_class.append("Severe Wind")
        numDetected+=1
        
    if "snow" in key_arr or "snowstorm" in key_arr:
        disaster_class.append("Snowstorm")
        numDetected+=1

    if "temperature" in key_arr and "low" in key_arr:
        disaster_class.append("Low Temperatures")
        numDetected+=1


    if "fire" in key_arr or "wildfire" in key_arr:
        disaster_class.append("Wildfire")
        numDetected+=1
        
    if numDetected==0:
        disaster_class.append("Undefined")
        
    if "http" in key_arr or "al" in key_arr or "reference" in key_arr:
        disaster_class = ["References"]
    
    return disaster_class

In [19]:
# Nice, now we can iterate through all files...
import re

df = pd.DataFrame(columns=['File', 'Page', 'Text','Classes','Places','Actions'])
acPage = 0

with os.scandir(pdf_docs_path) as entries:
    for entry in entries:
        if entry.name != ".DS_Store":
        #if entry.name == "81363.pdf":
            # Extract text and add to the datastore
            document = entry.name
            one_pdf_path = os.path.join(pdf_docs_path, document)
            
            with open(one_pdf_path, "rb") as f:
                try:
                    pdf = pdftotext.PDF(f)
                    i=0
                    for page in pdf:
                        i+=1
                        #contents = page.replace("\n", "")
                        contents = re.sub(r"[^a-zA-Z0-9:.,!?%$@]+", ' ', page)
                        if contents != "":
                            acPage+=1
                            impSents, places = impSentenceExtractor(contents)
                            keywords, wordList = frequentClimateWordsExtractor(contents)
                            disasterClass = disasterType(wordList)
                            
                            df.loc[acPage] = [document,i,contents,disasterClass,places,impSents]
                except:
                    print("Error on document",document)
            
            

In [17]:
df

,File,Page,Text,Classes,Places,Actions
1,81363.pdf,1,Climate Change Mitigation: a Strategic Approac...,[Mitigation],[],[]
2,81363.pdf,2,Toronto and Region Conservation TRCA has beco...,"[Carbon Neutrality, Climate Change Adaptation,...","[Toronto, Region Conservation TRCA, www.trca.o...",[Getting to carbon neutral is one in a series ...
3,81363.pdf,3,An Overview of Getting to Carbon Neutral: A Gu...,"[Carbon Neutrality, Severe Wind]","[Toronto, Calgary, Austria, Tokyo, the City of...",[Getting to carbon neutral also provides: guid...
4,81363.pdf,4,Getting to Carbon Neutral does not dictate wh...,"[Carbon Neutrality, Mitigation]",[],[ getting to carbon neutral does not dictate w...
5,81363.pdf,5,Tens steps to carbon neutrality Getting to Ca...,"[Carbon Neutrality, Severe Wind]",[Toronto],"[Develop bold, ambitious building codes and mu..."
6,81363.pdf,6,fired generating stations emit large amounts ...,[Carbon Neutrality],[MUNTAG],[Using the muntag model getting to carbon neut...
7,81363.pdf,7,GHG Reduction Strategies Sector Description of...,[Severe Wind],[Canada],"[Reduce energy demand by: and electricity, the..."
8,81363.pdf,8,Sector Description of Approach Technological ...,"[Carbon Neutrality, Severe Wind]",[The City of Whitehorse s Wheel 2 Work],[Develop electricity from renewable options fo...
9,81363.pdf,9,Sector Description of Approach Technological ...,"[Carbon Neutrality, Mitigation, Severe Wind]","[Calgary, Alberta, CO2e]",[Support the 3rs and increase services they ma...
10,81363.pdf,10,A climate change action strategy need not be ...,[Carbon Neutrality],"[Canada, the City of Toronto, Concord]",[Getting to carbon neutral lets a municipality...


In [20]:
df.to_excel("all-pages-dec03.xlsx")